# Kaggle | Enefit - Predict Energy Behavior of Prosumers

## Data dict

In [66]:
import pandas as pd
import pathlib
from zipfile import ZipFile
import calendar
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import traceback
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.5f}'.format)

### Functions

In [70]:
def create_data_dictionary(df: pd.DataFrame):
    try:
    # identify colames and dtypes
        data_dict = df.dtypes
        data_dict = data_dict.reset_index()
        data_dict = data_dict.rename(columns = {'index': 'Feature', 0: 'Data Type'})
        print('identify colames and dtypes: done')
    # null count
        data_dict['null_count'] = [df[(df[feature].isnull()) | (df[feature] == '')].shape[0] for feature in data_dict['Feature']]
        data_dict['null_count_perc'] = data_dict['null_count']/(df.shape[0])
        print('null count: done')
    # data length and data example
        data_length_features = [feature for feature in data_dict['Feature']]
        data_length = pd.DataFrame({'Feature': data_length_features,
                                'length_occurancy': [None]*len(data_length_features),
                                'data_example': [None]*len(data_length_features)})
        data_length['length_occurancy'] = data_length['length_occurancy'].astype('object')
        for i, k in enumerate(data_dict['Feature']):
            if np.issubdtype(df[k].dtype, np.floating):
                example = list(df[k].describe().to_dict().values())[:4]
            else:
                example = df[k].value_counts()[:4].index.tolist()
            data_length.at[i, 'data_example'] = example
            if is_numeric_dtype(df[k]):
                data_length.at[i, 'length_occurancy'] = None
            else:
                index = df[k].str.len().value_counts().index.tolist()
                values = df[k].str.len().value_counts().values.tolist()
                result = list(zip(index, values))
                data_length.at[i, 'length_occurancy'] = result
                data_dict = data_dict.merge(data_length, on= ['Feature'])
        print('data length and data example: done')
        # unique values
        data_dict['unique_values'] = [round(df[feature].nunique()/df[feature].shape[0] , 4) for feature in data_dict['Feature']]
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return None
    return data_dict

# Read file

In [3]:
root = '/home/carolinefcg/Documents/GitHub/Kaggle/predict-energy-behavior-of-prosumers/'

In [71]:
train_raw = pd.read_csv(root+'train.csv')
train_dict = create_data_dictionary(train_raw)
train_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy,data_example,unique_values
0,county,int64,0,0.00000,None,"[0, 11, 7, 5]",0.00000
1,is_business,int64,0,0.00000,None,"[1, 0]",0.00000
2,product_type,int64,0,0.00000,None,"[3, 1, 0, 2]",0.00000
3,target,float64,528,0.00026,None,"[2017824.0, 274.8555600988986, 909.50237801986...",0.28020
4,is_consumption,int64,0,0.00000,None,"[0, 1]",0.00000
5,datetime,object,0,0.00000,"[(19, 2018352)]","[2022-11-27 12:00:00, 2022-11-17 03:00:00, 202...",0.00760
6,data_block_id,int64,0,0.00000,None,None,0.00030
7,row_id,int64,0,0.00000,None,None,1.00000
8,prediction_unit_id,int64,0,0.00000,None,None,0.00000


In [72]:
client_raw = pd.read_csv(root+'client.csv')
client_dict = create_data_dictionary(client_raw)
client_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy,data_example,unique_values
0,product_type,int64,0,0.00000,None,"[3, 1, 0, 2]",0.00010
1,county,int64,0,0.00000,None,"[0, 11, 7, 5]",0.00040
2,eic_count,int64,0,0.00000,None,"[14, 8, 12, 9]",0.01150
3,installed_capacity,float64,0,0.00000,None,"[41919.0, 1450.7714513705005, 2422.23312018854...",0.04660
4,is_business,int64,0,0.00000,None,"[1, 0]",0.00000
5,date,object,0,0.00000,"[(10, 41919)]","[2022-11-26, 2022-11-09, 2022-11-11, 2022-11-12]",0.01520
6,data_block_id,int64,0,0.00000,None,None,0.01520


In [73]:
e_prices_raw = pd.read_csv(root+'electricity_prices.csv')
e_prices_dict = create_data_dictionary(e_prices_raw)
e_prices_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy_x,data_example_x,length_occurancy_y,data_example_y,unique_values
0,forecast_date,object,0,0.00000,"[(19, 15286)]","[2021-09-01 00:00:00, 2022-10-30 20:00:00, 202...","[(19, 15286)]","[2021-09-01 00:00:00, 2022-10-30 20:00:00, 202...",1.00000
1,euros_per_mwh,float64,0,0.00000,None,None,None,"[15286.0, 157.06417571634177, 121.148624971588...",0.72120
2,origin_date,object,0,0.00000,None,None,"[(19, 15286)]","[2021-08-31 00:00:00, 2022-10-29 20:00:00, 202...",1.00000
3,data_block_id,int64,0,0.00000,None,None,None,None,0.04170


In [7]:
forecast_weather_raw = pd.read_csv(root+'forecast_weather.csv')
forecast_weather_dict = create_data_dictionary(forecast_weather_raw)
forecast_weather_dict

In [8]:
gas_prices_raw = pd.read_csv(root+'gas_prices.csv')
gas_prices_dict = create_data_dictionary(gas_prices_raw)
gas_prices_dict

In [9]:
hist_weather_raw = pd.read_csv(root+'historical_weather.csv')
hist_weather_dict = create_data_dictionary(hist_weather_raw)
hist_weather_dict

In [74]:
weather_station_to_county_mapping_raw = pd.read_csv(root+'weather_station_to_county_mapping.csv')
weather_station_to_county_mapping_raw_dict = create_data_dictionary(weather_station_to_county_mapping_raw)
weather_station_to_county_mapping_raw_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy,data_example,unique_values
0,county_name,object,63,0.56250,"[(8.0, 29), (11.0, 7), (7.0, 6), (13.0, 4), (9...","[Harjumaa, Pärnumaa, Võrumaa, Saaremaa]",0.13390
1,longitude,float64,0,0.00000,None,None,0.12500
2,latitude,float64,0,0.00000,None,None,0.07140
3,county,float64,63,0.56250,None,None,0.13390
